In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
import xgboost 
from neupy.estimators import mse
from datetime import date
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dta = pd.read_excel('./Data/Seasonal.xlsx',index_col='Date')
# dta['Jan'] = dta['Month']==1
dta.head()

,Death,Death_rate,Incidence,Incidence_rate,forecast,Year,Month,error_ARIMA
Date,,,,,,,,
2005-01-01,143,0.011001,114194,8.784966,NaN,2005,1,NaN
2005-02-01,201,0.015463,87112,6.701542,NaN,2005,2,NaN
2005-03-01,281,0.021617,130134,10.011233,NaN,2005,3,NaN
2005-04-01,374,0.028772,133924,10.302798,NaN,2005,4,NaN
2005-05-01,388,0.029849,121326,9.333632,NaN,2005,5,NaN


In [5]:
dataset = dta.loc['2006-02-1':'2014-12-01'][['Incidence_rate','forecast']]
x_train = dataset.loc[:'2012-12-01'][['forecast']]
y_train = dataset.loc[:'2012-12-01']['Incidence_rate']
x_test = dataset.loc['2013-01-01':'2013-12-01'][['forecast']]
y_test = dataset['2013-01-01':'2013-12-01']['Incidence_rate']

In [6]:
# fit model no training data
model = xgboost.XGBRegressor(objective='reg:linear')
model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [7]:
def try_std(n_estimators=20,max_depth=5,learning_rate=0.1):
    model = xgboost.XGBRegressor(objective='reg:linear',\
                                 n_estimators=n_estimators,\
                                 max_depth=max_depth,\
                                 learning_rate=learning_rate)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(mse(y_pred,y_test))

In [8]:
for n_estimators in range(20,40,10):
    for max_depth in range(5,20,5):
        for learning_rate in range(16,20,1):
            print(n_estimators,max_depth,learning_rate)
            try_std(n_estimators=n_estimators,max_depth=max_depth,learning_rate=learning_rate/100)
            print('--\n')

20 5 16
0.06683116710161184
--

20 5 17
0.05275614054684615
--

20 5 18
0.04821134690488436
--

20 5 19
0.0449733268147089
--

20 10 16
0.0674105271166958
--

20 10 17
0.0542256509677962
--

20 10 18
0.049287594243478554
--

20 10 19
0.04706428900988303
--

20 15 16
0.0674105271166958
--

20 15 17
0.0542256509677962
--

20 15 18
0.049287594243478554
--

20 15 19
0.049301545161351844
--

30 5 16
0.03959007891496715
--

30 5 17
0.03877866686882072
--

30 5 18
0.03467367947844699
--

30 5 19
0.03976706077985308
--

30 10 16
0.05661406068269989
--

30 10 17
0.055293654616470074
--

30 10 18
0.05204155731953227
--

30 10 19
0.053598841462452994
--

30 15 16
0.058293067703397324
--

30 15 17
0.05527182415077254
--

30 15 18
0.056866130467808834
--

30 15 19
0.05728085536896176
--



In [9]:
try_std(n_estimators=30,max_depth=5,learning_rate=0.18)

0.03467367947844699


In [10]:
# fit model no training data
model = xgboost.XGBRegressor(objective='reg:linear',n_estimators=30,max_depth=5,learning_rate=0.18)
model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.18, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=30, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [15]:
dataset['XGBoost'] = model.predict(dataset[['forecast']])

In [16]:
mse(dataset['XGBoost'],dataset['Incidence_rate'])

0.044265275418226516

In [ ]:
data_train.to_excel('Seasonal_XGBoost.xlsx')